<div><img style="float: right; width: 120px; vertical-align:middle" src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/EU_Informatica/ETSI%20SIST_INFORM_COLOR.png" alt="ETSISI logo" />

# Implementado redes recurrentes simples<a id="top"></a>

<i>Última actualización: 2024-03-07</small></i></div>
***

## Introducción

Las redes neuronales recurrentes (RNN, del inglés _recurrent neural networks_) son un tipo de red neuronal que se utiliza para procesar datos secuenciales. A diferencia de las redes neuronales tradicionales, las RNN tienen conexiones entre los nodos que permiten que la información pase de un paso al siguiente. Esto las hace especialmente útiles para procesar datos que tienen una estructura temporal o secuencial, como las series temporales, el habla y el texto.

La idea básica de las RNN es utilizar la salida del paso anterior como entrada para el paso actual. Esto crea un bucle de retroalimentación que permite a la red mantener información sobre la secuencia que ha procesado hasta el momento. En su forma más simple, se implementan mediante unidades recurrentes simples (SRU). Las SRU son a las RNN lo que las neuronas son a las redes neuronales tradicionales. La única diferencia es que la salida de la red se concatena con la entrada, de modo que la salida anterior forma parte de la entrada actual.

## Objetivos

En este _notebook_ vamos a implementar una red neuronal recurrente para resolver el problema [`mnist`](http://yann.lecun.com/exdb/mnist/). En realidad lo haremos más como un ejercicio que como un problema de aplicación real, ya que técnicamente el `mnist` es un problema de reconocimiento de imágenes. Sin embargo, como veremos, las RNNs, leyendo las filas de arriba a abajo de la imagen de los números son capaces de conseguir resultados comparables a los de las Redes de Convolución (CNNs).

Al final habremos aprendido a:

- Crear y entrenar un modelo recurrente para resolver problemas de clasificación utilizando, para ello, una SRU.
- Apilar dos o más SRUs para hacer Redes Recurrentes multicapa, aumentando así la potencia de estas redes.

## Bibliotecas y configuración

A continuación importaremos las bibliotecas que se utilizarán a lo largo del cuaderno.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

También configuraremos algunos parámetros para adaptar la presentación gráfica.

In [ ]:
%matplotlib inline
plt.rcParams.update({'figure.figsize': (20, 6),'figure.dpi': 64})
plt.style.use('ggplot')

***

## Descarga y preprocesamiento de datos

Comenzamos como en el resto de _notebooks_, descargando y preparando el conjunto `mnist` para nuestra tarea.

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255, x_test / 255

print(f'Training shape: {x_train.shape} input, {y_train.shape} output')
print(f'Test shape:     {x_test.shape} input, {y_test.shape} output')

## Modelo basado en una capa de SRU

La primera capa será una SRU (implementada en keras como `layers.SimpleRNN`) de 10 «unidades» (una salida de dimensión 10) y cuya entrada son las dimensiones de la imagen (es decir, $28 \times 28$).

La segunda capa será una densa con activación softmax para las 10 posibles salidas. Esto significa que los 10 valores de salida de nuestra `SimpleRNN` se conectarán a 10 neuronas de nuestra capa `Dense` para un total de $10 \cdot 10 + 10$ conexiones.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(units=8, input_shape=(28, 28)),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.summary()

Aunque estamos indicando una entrada de $28 \times $28 no estamos ofreciendo una entrada de 28 filas por 28 columnas; estamos ofreciendo una entrada de 28 elementos de una secuencia, cada uno de tamaño 28. Es decir, nuestra red va a recibir primero la primera fila de la imagen, luego la segunda, luego la tercera, y así sucesivamente hasta la fila número 28.

Por último, vamos a compilar el modelo creado con la función de pérdida que corresponde a este tipo de problema con un optimizador de descenso de gradiente estocástico y vamos a añadir la métrica _exaccuracy_ para ver cómo evoluciona este entrenamiento.

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['sparse_categorical_accuracy'],
)

### Entrenamiento del modelo

Por último, entrenaremos nuestra red durante $25$ epochs. Cuidado con el tiempo de entrenamiento, ya que en el caso de las redes recurrentes es bastante lento.

In [ ]:
history = model.fit(x_train, y_train, epochs=25)

Veamos cómo ha evolucionado la formación.

In [ ]:
pd.DataFrame(history.history).plot()
plt.yscale('log')
plt.xlabel('Epoch num.')
plt.show()

Podemos ver que están lejos de las precisiones conseguidas con otras técnicas de aprendizaje profundo, especialmente si las comparamos con las redes convolucionales. Pero como hemos dicho, se trata de un ejemplo de implementación, no de un caso de uso concreto.

### Clasificación de dígitos

Vamos a hacer algunas inferencias sobre el conjunto de datos de prueba. Veremos que nuestro modelo falla significativamente más que en otros casos.

In [ ]:
# Predict some of the test digits
ROWS, COLS = 5, 5
IMAGES = ROWS * COLS
ŷ_test = np.argmax(model.predict(x_test[:IMAGES], verbose=0), axis=1)
# And plot them
fig = plt.figure(figsize=(15, 15))
for i, (x, y, ŷ) in enumerate(zip(x_test[:IMAGES], y_test[:IMAGES], ŷ_test), 1):
    ax = fig.add_subplot(ROWS, COLS, i)
    ax.imshow(x, cmap='Greens' if y == ŷ else 'Reds')
    ax.set_title(f'Expected: {y}, predicted: {ŷ}')
    ax.grid(False)
plt.tight_layout()

## Conclusiones

Hemos aprendido a implementar una RNN simple en Keras, construyendo un modelo sencillo para clasificar las imágenes `mnist` en sus correspondientes etiquetas, demostrando cómo compilar y entrenar el modelo, y cómo evaluar su rendimiento utilizando la exactitud (_accuracy_) como métrica. En realidad, hemos hecho lo que hasta ahora, pero con redes recurrentes, y hemos visto que es prácticamente igual.

Pero al menos hemos aprendido que se puede hacer y que es fácil. Est _notebook_ nos sirve como punto de partida para implementar modelos RNN más complejos en Keras para tareas como la predicción de series temporales o el procesamiento del lenguaje natural.

***

<div><img style="float: right; width: 120px; vertical-align:top" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" alt="Creative Commons by-nc-sa logo" />

[Volver al inicio](#top)

</div>